In [1]:
import os
import pandas as pd
import numpy as np
import plotly.express as px

# Definición de funciones

### Conversión de columna tipo fecha y transformación de nombre de unidad por ciudad

In [315]:
def datosFechas(main_path, name_file, name_column_fecha):
    """
    main_path: Nombre de ruta principal
    name_file: Nombre de archivo
    name_column_fecha: Nombre de la columna que contiene las fechas
    
    La función trata la columna fecha, convierte en datos de fecha devolviendo columnas separadas por año, mes y día.
    Además crea una nueva columna con el nombre correspondiente a la ciudad de la estación del SACER tomando como dato,
    el nombre de la unidad.
    """
    
    df = pd.read_csv((os.path.join(main_path, name_file)),sep=',')
    
    df.loc[df['Unidad'] =="scn-l06", 'Ciudad'] = 'Quito' 
    df.loc[df['Unidad'] =="scn-l01", 'Ciudad'] = 'Tulcan'
    df.loc[df['Unidad'] =="scn-l02", 'Ciudad'] = 'Ibarra' 
    df.loc[df['Unidad'] =="scn-l03", 'Ciudad'] = 'Esmeraldas'
    df.loc[df['Unidad'] =="scn-l05", 'Ciudad'] = 'Nueva_Loja' 
    df.loc[df['Unidad'] =="scd-l01", 'Ciudad'] = 'Riobamba'
    df.loc[df['Unidad'] =="scd-l02", 'Ciudad'] = 'Ambato' 
    df.loc[df['Unidad'] =="scd-l03", 'Ciudad'] = 'Puyo'
    df.loc[df['Unidad'] =="scm-l01", 'Ciudad'] = 'Manta' 
    df.loc[df['Unidad'] =="scm-l02", 'Ciudad'] = 'Sto_Domingo'
    df.loc[df['Unidad'] =="scc-l01", 'Ciudad'] = 'Santa_Elena' 
    df.loc[df['Unidad'] =="scc-l02", 'Ciudad'] = 'Guayaquil'
    df.loc[df['Unidad'] =="scc-l03", 'Ciudad'] = 'Quevedo' 
    df.loc[df['Unidad'] =="scc-l05", 'Ciudad'] = 'Taura'
    df.loc[df['Unidad'] =="scs-l01", 'Ciudad'] = 'Zamora' 
    df.loc[df['Unidad'] =="scs-l02", 'Ciudad'] = 'Loja'
    df.loc[df['Unidad'] =="scs-l03", 'Ciudad'] = 'Canar' 
    df.loc[df['Unidad'] =="scs-l04", 'Ciudad'] = 'Macas'
    df.loc[df['Unidad'] =="scs-l05", 'Ciudad'] = 'Cuenca' 
    df.loc[df['Unidad'] =="scc-l04", 'Ciudad'] = 'Machala'
    df.loc[df['Unidad'] =="erm-l01", 'Ciudad'] = 'San_Cristobal' 
    df.loc[df['Unidad'] =="erm-l02", 'Ciudad'] = 'Santa_Cruz'
    
    df['Date'] = pd.to_datetime(df['Fecha'])
    df['Year']= df['Date'].dt.year
    df['Month']= df['Date'].dt.month
    df['Day']= df['Date'].dt.day
    
    df.drop(columns=['Fecha','Unidad'], inplace=True)
    
    return df

### Creación de un DataFrame con los con el número de meses que operó una frecuencia TV

In [316]:
def outDataFrame_tv(ciudad, frecuencia, Opera, No_opera):
    """
    ciudad: Nombre de la ciudad,
    frecuenica: frecuencia a evaluar,
    Opera: Número de meses en que operó una frecuencia,
    No_opera: Número de meses en que no noperó una frecuencia.
    
    La función toma los valores asignados por el bucle y devuelve un DataFrame con vectores anteriormente especificados.
    """
    datos = [{
        'Ciudad': ciudad,
        'Frecuencia': frecuencia,
        'Nro. Meses Opera': Opera,
        'Nro. Meses No Opera': No_opera
    }]
    df = pd.DataFrame(datos)
    return df

# Funciones para Banda I de TV 

### Creación de un DataFrame con las frecuencias que operan en Banda I de TV analógica

In [317]:
def frecBandaI (data):
    """
    data: DataFrame.
    
    La función devuelve un DataFrame con las frecuencias de operación en Banda I de TV analógica
    """
    list_banda_I = []
    banda_I = [55.25, 59.75, 61.25, 65.75, 67.25, 71.75, 77.25, 81.75, 83.25, 87.75]
    
    for frec, group_frec in data.groupby('Frecuencia'):
        for value in banda_I:
            if frec == value:
                list_banda_I.append(group_frec)

    df_banda_I = pd.concat(list_banda_I)
    
    return df_banda_I

### Creación de un DataFramae con el número de meses que operó una frecuencia de Banda I de TV analógica

In [318]:
def mesesOperaFrecuenciaTV_banda_I(data, ciudad):
    """
    data: DataFrame del reporte de TV promedio mensual,
    ciudad: Nombre de la ciudad a evaluar
    
    La función devuelve un DataFrame con el número de meses que operó y no operó una 
    frecuencia pertenenciente a la Banda I de TV analógica.
    """
    
    # método frecBandaI() para obtener un df con las frecuencias de la banda I de TV analógica
    df_banda_I = frecBandaI(data)
    appended_data = []

    for frec, group_banda_I in df_banda_I.groupby('Frecuencia'):
        No_opera = 0
        Opera = 0

        for value in group_banda_I['Intensidad avg']:
            if value >= 68:
                Opera += 1
            else:
                No_opera += 1
        
        # método outDataFrame_TV() para crear un nuevo DF con el número de meses que opera una frecuencia de TV        
        df = outDataFrame_tv(ciudad, frec, Opera, No_opera)
        appended_data.append(df)

    appended_data = pd.concat(appended_data)
    appended_reset_index = appended_data.reset_index(drop=True, inplace=False)
    
    return appended_reset_index

# Funciones para Banda III de TV

In [319]:
def frecBandaIII (data):
    """
    data: DataFrame.
    
    La función devuelve un DataFrame con las frecuencias de operación en Banda III de TV analógica
    """
    list_banda_III = []
    banda_III = [175.25, 179.75, 181.25, 185.75, 187.25, 191.75, 193.25, 197.75, 199.25, 203.75,
                 205.25, 209.75, 211.25, 215.75]
    
    for frec, group_frec in data.groupby('Frecuencia'):
        for value in banda_III:
            if frec == value:
                list_banda_III.append(group_frec)

    df_banda_III = pd.concat(list_banda_III)
    
    return df_banda_III

In [320]:
def mesesOperaFrecuenciaTV_banda_III(data, ciudad):
    """
    data: DataFrame del reporte de TV promedio mensual,
    ciudad: Nombre de la ciudad a evaluar
    
    La función devuelve un DataFrame con el número de meses que operó y no operó una 
    frecuencia pertenenciente a la Banda III de TV analógica.
    """
    
    # método frecBandaI() para obtener un df con las frecuencias de la banda I de TV analógica
    df_banda_III = frecBandaIII(data)
    appended_data = []

    for frec, group_banda_III in df_banda_III.groupby('Frecuencia'):
        No_opera = 0
        Opera = 0

        for value in group_banda_III['Intensidad avg']:
            if value >= 71:
                Opera += 1
            else:
                No_opera += 1
        
        # método outDataFrame_TV() para crear un nuevo DF con el número de meses que opera una frecuencia de TV        
        df = outDataFrame_tv(ciudad, frec, Opera, No_opera)
        appended_data.append(df)

    appended_data = pd.concat(appended_data)
    appended_reset_index = appended_data.reset_index(drop=True, inplace=False)
    
    return appended_reset_index

# Funciones para Banda IV de TV

In [321]:
def frecBandaIV (data):
    """
    data: DataFrame.
    
    La función devuelve un DataFrame con las frecuencias de operación en Banda III de TV analógica
    """
    list_banda_IV = []
    banda_IV = [471.25, 475.75, 477.25, 481.75, 483.25, 487.75, 513.25, 517.75, 519.25, 523.75,
                525.25, 529.75, 531.25, 535.75, 537.25, 541.75, 543.25, 547.75, 549.25, 553.75,
                555.25, 559.75, 561.25, 565.75, 567.25, 571.75, 573.25, 577.75, 579.25, 583.75,
                585.25, 589.75, 591.25, 595.75, 597.25, 601.75, 603.25, 607.75, 609.25, 613.75,
                615.25, 619.75, 621.25, 625.75, 627.25, 631.75, 633.25, 637.75, 639.25, 643.75]
    
    for frec, group_frec in data.groupby('Frecuencia'):
        for value in banda_IV:
            if frec == value:
                list_banda_IV.append(group_frec)

    df_banda_IV = pd.concat(list_banda_IV)
    
    return df_banda_IV

In [322]:
def mesesOperaFrecuenciaTV_banda_IV(data, ciudad):
    """
    data: DataFrame del reporte de TV promedio mensual,
    ciudad: Nombre de la ciudad a evaluar
    
    La función devuelve un DataFrame con el número de meses que operó y no operó una 
    frecuencia pertenenciente a la Banda IV de TV analógica.
    """
    
    # método frecBandaI() para obtener un df con las frecuencias de la banda I de TV analógica
    df_banda_IV = frecBandaIV(data)
    appended_data = []

    for frec, group_banda_IV in df_banda_IV.groupby('Frecuencia'):
        No_opera = 0
        Opera = 0

        for value in group_banda_IV['Intensidad avg']:
            if value >= 74:
                Opera += 1
            else:
                No_opera += 1
        
        # método outDataFrame_TV() para crear un nuevo DF con el número de meses que opera una frecuencia de TV        
        df = outDataFrame_tv(ciudad, frec, Opera, No_opera)
        appended_data.append(df)

    appended_data = pd.concat(appended_data)
    appended_reset_index = appended_data.reset_index(drop=True, inplace=False)
    
    return appended_reset_index

# Funciones para Banda V de TV

In [323]:
def frecBandaV (data):
    """
    data: DataFrame.
    
    La función devuelve un DataFrame con las frecuencias de operación en Banda III de TV analógica
    """
    list_banda_V = []
    banda_V = [645.25, 649.75, 651.25, 655.75, 657.25, 661.75, 663.25, 667.75, 669.25, 673.75,
               675.25, 679.75, 681.25, 685.75, 687.25, 691.75, 693.25, 697.75]
    
    for frec, group_frec in data.groupby('Frecuencia'):
        for value in banda_V:
            if frec == value:
                list_banda_V.append(group_frec)

    df_banda_V = pd.concat(list_banda_V)
    
    return df_banda_V

In [324]:
def mesesOperaFrecuenciaTV_banda_V(data, ciudad):
    """
    data: DataFrame del reporte de TV promedio mensual,
    ciudad: Nombre de la ciudad a evaluar
    
    La función devuelve un DataFrame con el número de meses que operó y no operó una 
    frecuencia pertenenciente a la Banda V de TV analógica.
    """
    
    # método frecBandaI() para obtener un df con las frecuencias de la banda I de TV analógica
    df_banda_V = frecBandaV(data)
    appended_data = []

    for frec, group_banda_V in df_banda_V.groupby('Frecuencia'):
        No_opera = 0
        Opera = 0

        for value in group_banda_V['Intensidad avg']:
            if value >= 74:
                Opera += 1
            else:
                No_opera += 1
        
        # método outDataFrame_TV() para crear un nuevo DF con el número de meses que opera una frecuencia de TV        
        df = outDataFrame_tv(ciudad, frec, Opera, No_opera)
        appended_data.append(df)

    appended_data = pd.concat(appended_data)
    appended_reset_index = appended_data.reset_index(drop=True, inplace=False)
    
    return appended_reset_index

# Función que concatena todas las funciones para meses de operación de las bandas de TV

In [325]:
def estadoOperacionTV(data, ciudad):
    """
    data = DataFrame con las conversiones de fechas y nombre de ciudad,
    ciudad = Nombre de la ciudad a evaluar
    
    La función devuelve un DataFrame con el número de meses de operación de una ciudad en todas las bandas de TV analógica.
    Se debe tomar en cuenta que se usan las bandas de Video y Audio.
    """
    appended_data = []
    
    data_banda_I = mesesOperaFrecuenciaTV_banda_I(data=data, ciudad=ciudad)
    appended_data.append(data_banda_I)
    
    data_banda_III = mesesOperaFrecuenciaTV_banda_III(data=data, ciudad=ciudad)
    appended_data.append(data_banda_III)
    
    data_banda_IV = mesesOperaFrecuenciaTV_banda_IV(data=data, ciudad=ciudad)
    appended_data.append(data_banda_IV)
    
    data_banda_V = mesesOperaFrecuenciaTV_banda_V(data=data, ciudad=ciudad)
    appended_data.append(data_banda_V)
    
    appended_data = pd.concat(appended_data)
    appended_data_reset_index = appended_data.reset_index(drop=True, inplace=False)
    
    return appended_data_reset_index

# CZO2

## Data wrangling

#### Carga de archivo principal

In [78]:
main_path = 'Proyecto_git/Entrada/'
name_file = 'Unidad_scn-l01_2021-01-01-2022-01-01.csv'

In [79]:
data = pd.read_csv(os.path.join(main_path, name_file), sep=',')
data.head(2)

,Fecha,Frecuencia,Intensidad avg,Ancho de banda avg,Unidad
0,2021-01-01 00:00:00-05:00,55.25,73.17,179.16,scn-l01
1,2021-01-01 00:00:00-05:00,59.75,62.67,120.62,scn-l01


#### Dando formato Fecha a la columna Fecha y obtención de la ciudad a evaluar

In [80]:
# Uso del método datosFechas() para tratar las fechas y obtener el nombre de la ciudad de la estación

data1 = datosFechas(main_path=main_path, name_file=name_file, name_column_fecha='Fecha')
data1.head(2)

,Frecuencia,Intensidad avg,Ancho de banda avg,Ciudad,Date,Year,Month,Day
0,55.25,73.17,179.16,Tulcan,2021-01-01 00:00:00-05:00,2021,1,1
1,59.75,62.67,120.62,Tulcan,2021-01-01 00:00:00-05:00,2021,1,1


In [81]:
for city, group_data in data1.groupby('Ciudad'):
    print(city)

Esmeraldas
Ibarra
Nueva_Loja
Quito
Tulcan


#### Filtrado de datos por ciudad

In [82]:
data2 = data1.groupby('Ciudad').get_group('Quito')
data2.head()

,Frecuencia,Intensidad avg,Ancho de banda avg,Ciudad,Date,Year,Month,Day
3854,55.25,96.51,103.83,Quito,2021-01-01 00:00:00-05:00,2021,1,1
3855,59.75,85.05,94.79,Quito,2021-01-01 00:00:00-05:00,2021,1,1
3856,61.25,74.25,847.35,Quito,2021-01-01 00:00:00-05:00,2021,1,1
3857,65.75,70.92,123.19,Quito,2021-01-01 00:00:00-05:00,2021,1,1
3858,67.25,102.82,113.81,Quito,2021-01-01 00:00:00-05:00,2021,1,1


In [83]:
# Agrupando por año y mostrando el conteo de los valores de la columna mes

data2.groupby('Year')['Month'].count()

Year
2021    1104
Name: Month, dtype: int64

In [84]:
for frec, group_frec in data2.groupby('Frecuencia'):
    print (frec)

55.25
59.75
61.25
65.75
67.25
71.75
77.25
81.75
83.25
87.75
175.25
179.75
181.25
185.75
187.25
191.75
193.25
197.75
199.25
203.75
205.25
209.75
211.25
215.75
513.25
517.75
519.25
523.75
525.25
529.75
531.25
535.75
537.25
541.75
543.25
545.14
547.75
549.25
553.75
555.25
559.75
561.25
565.75
567.25
569.14
571.75
573.25
577.75
579.25
581.14
583.75
585.25
589.75
591.25
593.14
595.75
597.25
601.75
603.25
605.14
607.75
609.25
613.75
615.25
619.75
621.25
623.14
625.75
627.25
631.75
633.25
635.14
637.75
639.25
643.75
645.25
647.14
649.75
651.25
655.75
657.25
659.14
661.75
663.25
667.75
669.25
671.14
673.75
675.25
679.75
681.25
685.75


De los datos obtenidos se observa la existencia de portadoras de Video y Audio para las 4 bandas de TV:
* Banda I: 55.25, 59.75
* Banda III: 175.25, 179.75
* Banda IV: 513.25, 517.75
* Banda V: 645.25, 647.14

## Número de meses de operación TV de Tulcán

In [35]:
tulcan_tv = estadoOperacionTV(data=data1, ciudad='Tulcan')
tulcan_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Tulcan,55.25,8,4
1,Tulcan,59.75,0,12
2,Tulcan,61.25,12,0
3,Tulcan,65.75,0,12
4,Tulcan,67.25,12,0


## Número de meses de operación TV de Ibarra

In [55]:
ibarra_tv = estadoOperacionTV(data=data2, ciudad='Ibarra')
ibarra_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Ibarra,55.25,0,12
1,Ibarra,59.75,0,12
2,Ibarra,61.25,4,8
3,Ibarra,65.75,0,12
4,Ibarra,67.25,0,12


## Número de meses de operación TV de Esmeraldas

In [65]:
esmeraldas_tv = estadoOperacionTV(data=data2, ciudad='Esmeraldas')
esmeraldas_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Esmeraldas,55.25,0,11
1,Esmeraldas,59.75,0,11
2,Esmeraldas,61.25,0,11
3,Esmeraldas,65.75,0,11
4,Esmeraldas,67.25,11,0


## Número de meses de operación TV de Nueva_Loja

In [77]:
nueva_loja_tv = estadoOperacionTV(data=data2, ciudad='Nueva_Loja')
nueva_loja_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Nueva_Loja,55.25,0,12
1,Nueva_Loja,59.75,0,12
2,Nueva_Loja,61.25,0,12
3,Nueva_Loja,65.75,0,12
4,Nueva_Loja,67.25,0,12


## Número de meses de operación TV de Quito

In [85]:
quito_tv = estadoOperacionTV(data=data2, ciudad='Quito')
quito_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Quito,55.25,12,0
1,Quito,59.75,12,0
2,Quito,61.25,12,0
3,Quito,65.75,12,0
4,Quito,67.25,12,0


In [91]:
list_czo2 = []
list_czo2.append(tulcan_tv)
list_czo2.append(ibarra_tv)
list_czo2.append(esmeraldas_tv)
list_czo2.append(nueva_loja_tv)
list_czo2.append(quito_tv)

In [92]:
czo2_tv = pd.concat(list_czo2)

In [93]:
czo2_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Tulcan,55.25,8,4
1,Tulcan,59.75,0,12
2,Tulcan,61.25,12,0
3,Tulcan,65.75,0,12
4,Tulcan,67.25,12,0


In [95]:
name_path = 'Proyecto_git/Intermedio/'
name_file = 'czo2_tv.xlsx'

In [96]:
czo2_tv.to_excel(os.path.join(name_path,name_file))

# CZO3

In [136]:
main_path = '05_ReportesPromedio_TV/2022/CZ03/CSV files/'
name_file = 'Unidad_scd-l03_2021-01-01-2022-01-01.csv'

In [137]:
data = datosFechas(main_path=main_path, name_file=name_file, name_column_fecha='Fecha')
data.head()

,Frecuencia,Intensidad avg,Ancho de banda avg,Ciudad,Date,Year,Month,Day
0,55.25,91.21,82.25,Riobamba,2021-01-01 00:00:00-05:00,2021,1,1
1,67.25,63.44,74.17,Riobamba,2021-01-01 00:00:00-05:00,2021,1,1
2,77.25,77.91,77.48,Riobamba,2021-01-01 00:00:00-05:00,2021,1,1
3,55.25,91.38,83.04,Riobamba,2021-02-01 00:00:00-05:00,2021,2,1
4,67.25,63.26,74.77,Riobamba,2021-02-01 00:00:00-05:00,2021,2,1


In [138]:
data.groupby('Ciudad').count()

,Frecuencia,Intensidad avg,Ancho de banda avg,Date,Year,Month,Day
Ciudad,,,,,,,
Ambato,304,304,304,304,304,304,304
Puyo,649,649,649,649,649,649,649
Riobamba,273,273,273,273,273,273,273


In [139]:
data.groupby('Year')['Month'].count()

Year
2021    1226
Name: Month, dtype: int64

In [146]:
data1 = data.groupby('Ciudad').get_group('Riobamba')

In [147]:
for frec, group_data in data1.groupby('Frecuencia'):
    print(frec)

55.25
67.25
77.25
88.0
175.25
181.25
187.25
199.25
211.25
500.07
500.15
500.22
500.32
500.82
501.1
501.18
501.26
501.72
501.86
501.9
501.91
501.92
501.93
502.21
502.23
502.67
502.85
505.13
505.15
505.16
510.03
510.86
513.25
525.25
537.25
549.25
561.25
573.25
585.25
597.25
615.25
675.25


De los datos obtenidos se observa lo siguiente:

* Banda I: Existencia solo de portadoras de Video 
    * 55.25, 67.25, 77.25
    * 88.0 (Extremo derecho del rango de frecuencias para el canal 6)
* Banda III: Existencia solo de portadoras de Video
    * 175.25, 181.25, 187.25, 199.25, 211.25
* Banda IV: Se observan extraños listados distintos a los rangos de frecuencias establecidos en la norma y frecuencias de portadoras de video.
    * 500.07, 500.15, 500.22, 500.32, 500.82 (Extaño listado, podrían ser monitoreos de frecuencias no autorizadas)
    * 501.1, 501.18, 501.26, 501.72, 501.86, 501.9, 501.91, 501.92, 501.93 (Extraño listado, podrían ser monitoreos de frecuencias no autorizadas).
    * 513.25, 525.25, 537.25, 549.25, 561.25, 573.25 $\cdots$
* Banda V: Existencia solo de portadoras de Video.
    * 675.25

## Número de meses de operación TV Riobamba

In [148]:
riobamba_tv = estadoOperacionTV(data=data1, ciudad='Riobamba')
riobamba_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Riobamba,55.25,14,0
1,Riobamba,67.25,0,14
2,Riobamba,77.25,14,0
3,Riobamba,175.25,14,0
4,Riobamba,181.25,14,0


* Para Riobamaba se observan posibles datos duplicados en los promedios.
    * Mayo y Junio

In [220]:
riobamba_tv.to_excel('06_Archivos_intermedios/Duplicados/riobamba_tv_v1.xlsx')

### Datos Riobamba - wrangling

In [257]:
main_path = '05_ReportesPromedio_TV/2022/CZ03/Excel files/'
name_file = 'Unidad_scd-l01_2021-01-01-2022-01-01.xlsx'

In [258]:
data = pd.read_excel(os.path.join(main_path, name_file), index_col=False)
data.head()

,Fecha,Frecuencia,Intensidad avg,Ancho de banda avg,Unidad
0,2021-01-01 00:00:00-05:00,55.25,91.21,82.25,scd-l01
1,2021-01-01 00:00:00-05:00,67.25,63.44,74.17,scd-l01
2,2021-01-01 00:00:00-05:00,77.25,77.91,77.48,scd-l01
3,2021-02-01 00:00:00-05:00,55.25,91.38,83.04,scd-l01
4,2021-02-01 00:00:00-05:00,67.25,63.26,74.77,scd-l01


In [259]:
df.loc[df['Unidad'] =="scd-l01", 'Ciudad'] = 'Riobamba'

In [228]:
data.head()

,Fecha,Frecuencia,Intensidad avg,Ancho de banda avg,Unidad,Ciudad
0,2021-01-01 00:00:00-05:00,55.25,91.21,82.25,scd-l01,Riobamba
1,2021-01-01 00:00:00-05:00,67.25,63.44,74.17,scd-l01,Riobamba
2,2021-01-01 00:00:00-05:00,77.25,77.91,77.48,scd-l01,Riobamba
3,2021-02-01 00:00:00-05:00,55.25,91.38,83.04,scd-l01,Riobamba
4,2021-02-01 00:00:00-05:00,67.25,63.26,74.77,scd-l01,Riobamba


In [229]:
data['Date'] = pd.to_datetime(data['Fecha'])
data['Year']= data['Date'].dt.year
data['Month']= data['Date'].dt.month
data['Day']= data['Date'].dt.day
data.drop(columns=['Fecha','Unidad'], inplace=True)

In [230]:
data.head()

,Frecuencia,Intensidad avg,Ancho de banda avg,Ciudad,Date,Year,Month,Day
0,55.25,91.21,82.25,Riobamba,2021-01-01 00:00:00-05:00,2021,1,1
1,67.25,63.44,74.17,Riobamba,2021-01-01 00:00:00-05:00,2021,1,1
2,77.25,77.91,77.48,Riobamba,2021-01-01 00:00:00-05:00,2021,1,1
3,55.25,91.38,83.04,Riobamba,2021-02-01 00:00:00-05:00,2021,2,1
4,67.25,63.26,74.77,Riobamba,2021-02-01 00:00:00-05:00,2021,2,1


In [231]:
riobamba_tv = estadoOperacionTV(data=data, ciudad='Riobamba')
riobamba_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Riobamba,55.25,12,0
1,Riobamba,67.25,0,12
2,Riobamba,77.25,12,0
3,Riobamba,175.25,12,0
4,Riobamba,181.25,12,0


## Número de meses de operación TV Ambato

In [142]:
ambato_tv = estadoOperacionTV(data=data1, ciudad='Ambato')
ambato_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Ambato,55.25,0,2
1,Ambato,67.25,12,0
2,Ambato,77.25,12,0
3,Ambato,175.25,11,0
4,Ambato,181.25,12,0


## Número de meses de operación TV Puyo

In [145]:
puyo_tv = estadoOperacionTV(data=data1, ciudad='Puyo')
puyo_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Puyo,55.25,0,2
1,Puyo,67.25,11,3
2,Puyo,77.25,1,13
3,Puyo,175.25,0,14
4,Puyo,181.25,4,10


* Para Puyo se observan posibles datos duplicados en los promedios
    * Febrero, Junio

In [232]:
puyo_tv.to_excel('06_Archivos_intermedios/Duplicados/puyo_tv_v1.xlsx')

### Datos Puyo - wrangling

In [260]:
name_file = '06_Archivos_intermedios/Duplicados/Unidad_scd-l03_2021-01-01-2022-01-01_v1.xlsx'

In [266]:
data = pd.read_excel(name_file, index_col=False)
data.head()

,Fecha,Frecuencia,Intensidad avg,Ancho de banda avg,Unidad
0,2021-01-01 00:00:00-05:00,55.25,91.21,82.25,scd-l01
1,2021-01-01 00:00:00-05:00,67.25,63.44,74.17,scd-l01
2,2021-01-01 00:00:00-05:00,77.25,77.91,77.48,scd-l01
3,2021-02-01 00:00:00-05:00,55.25,91.38,83.04,scd-l01
4,2021-02-01 00:00:00-05:00,67.25,63.26,74.77,scd-l01


In [267]:
data.groupby('Unidad').count()

,Fecha,Frecuencia,Intensidad avg,Ancho de banda avg
Unidad,,,,
scd-l01,273,273,273,273
scd-l02,304,304,304,304
scd-l03,618,618,618,618


In [268]:
data = data.groupby('Unidad').get_group('scd-l03')
data.head(2)

,Fecha,Frecuencia,Intensidad avg,Ancho de banda avg,Unidad
577,2021-01-01 00:00:00-05:00,67.25,25.07,90.22,scd-l03
578,2021-01-01 00:00:00-05:00,77.25,22.57,90.22,scd-l03


In [269]:
data.loc[data1['Unidad'] =="scd-l03", 'Ciudad'] = 'Puyo'

In [270]:
data.head(2)

,Fecha,Frecuencia,Intensidad avg,Ancho de banda avg,Unidad,Ciudad
577,2021-01-01 00:00:00-05:00,67.25,25.07,90.22,scd-l03,Puyo
578,2021-01-01 00:00:00-05:00,77.25,22.57,90.22,scd-l03,Puyo


In [271]:
data['Date'] = pd.to_datetime(data['Fecha'])
data['Year']= data['Date'].dt.year
data['Month']= data['Date'].dt.month
data['Day']= data['Date'].dt.day
data.drop(columns=['Fecha','Unidad'], inplace=True)

In [272]:
data.head()

,Frecuencia,Intensidad avg,Ancho de banda avg,Ciudad,Date,Year,Month,Day
577,67.25,25.07,90.22,Puyo,2021-01-01 00:00:00-05:00,2021,1,1
578,77.25,22.57,90.22,Puyo,2021-01-01 00:00:00-05:00,2021,1,1
579,67.25,29.33,90.17,Puyo,2021-02-01 00:00:00-05:00,2021,2,1
580,77.25,27.99,90.15,Puyo,2021-02-01 00:00:00-05:00,2021,2,1
581,67.25,73.22,83.43,Puyo,2021-03-01 00:00:00-05:00,2021,3,1


In [274]:
puyo_tv = estadoOperacionTV(data=data, ciudad='Puyo')
puyo_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Puyo,55.25,0,1
1,Puyo,67.25,10,2
2,Puyo,77.25,1,11
3,Puyo,175.25,0,12
4,Puyo,181.25,3,9


In [275]:
list_czo3 = []
list_czo3.append(ambato_tv)
list_czo3.append(riobamba_tv)
list_czo3.append(puyo_tv)
czo3_tv = pd.concat(list_czo3)
czo3_tv.reset_index(drop=True, inplace=True)

In [276]:
czo3_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Ambato,55.25,0,2
1,Ambato,67.25,12,0
2,Ambato,77.25,12,0
3,Ambato,175.25,11,0
4,Ambato,181.25,12,0


In [277]:
name_file = '06_Archivos_intermedios/czo3_tv.xlsx'
sheet_name = 'CZO3'

In [278]:
czo3_tv.to_excel(name_file, sheet_name=sheet_name)

# CZO4

In [122]:
main_path = '05_ReportesPromedio_TV/2022/CZ04/CSV files/'
name_file = 'Unidad_scm-l02_2021-01-01-2022-01-01.csv'

In [123]:
datos = datosFechas(main_path=main_path, name_file=name_file, name_column_fecha='Fecha')

In [125]:
datos.groupby('Ciudad').count()

,Frecuencia,Intensidad avg,Ancho de banda avg,Date,Year,Month,Day
Ciudad,,,,,,,
Manta,752,752,752,752,752,752,752
Sto_Domingo,683,683,683,683,683,683,683


In [155]:
data1 = datos.groupby('Ciudad').get_group('Sto_Domingo')
frec_data1=[]
for frec, group_data1 in data1.groupby('Frecuencia'):
    frec_data1.append(frec)
print(frec_data1)

[55.25, 61.25, 67.25, 77.25, 83.25, 87.9, 175.25, 181.25, 187.25, 193.25, 199.25, 205.25, 211.25, 513.25, 515.14, 519.25, 521.14, 525.25, 527.14, 531.25, 533.14, 537.25, 539.14, 543.25, 545.14, 549.25, 551.14, 555.25, 557.14, 561.25, 563.14, 567.25, 569.14, 573.25, 575.14, 579.25, 581.14, 585.25, 587.14, 591.25, 593.14, 597.25, 599.14, 603.25, 605.14, 615.25, 617.14, 621.25, 623.14, 627.25, 629.14, 633.25, 635.14, 639.25, 641.14, 645.25, 647.14, 651.25, 653.14, 657.25, 659.14, 663.25, 665.14, 669.25, 671.14, 675.25, 677.14, 681.25, 683.14]


De los datos obtenidos se observa lo siguiente:
* Banda I: solo portadoras de video.
    * 55.25, 61.25, 67.25, 77.25
* Banda III: solo portadoras de video.
    *  175.25, 181.25, 187.25, 193.25, 199.25, 205.25, 211.25
* Banda IV: portadora de video y frecuencia central.
    *  513.25, 515.14, 519.25, 521.14, 525.25, 527.14, 531.25, 533.14 $\cdots$
* Banda V: portadora de video y frecuencia central.
    * 645.25, 647.14, 651.25, 653.14, 657.25, 659.14 $\cdots$

In [156]:
data1.groupby('Year')['Month'].count()

Year
2021    683
Name: Month, dtype: int64

## Número de meses de operación TV Manta

In [333]:
manta_tv = estadoOperacionTV(data=data1, ciudad='Manta')
manta_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Manta,55.25,0,1
1,Manta,67.25,10,2
2,Manta,77.25,1,11
3,Manta,175.25,0,12
4,Manta,181.25,3,9


## Número de meses de operación TV Sto_Domingo

In [157]:
sto_domingo_tv = estadoOperacionTV(data=data1, ciudad='Sto_Domingo')
sto_domingo_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Sto_Domingo,55.25,0,10
1,Sto_Domingo,61.25,0,10
2,Sto_Domingo,67.25,9,1
3,Sto_Domingo,77.25,9,1
4,Sto_Domingo,83.25,0,10


In [335]:
list_czo4 = []
list_czo4.append(manta_tv)
list_czo4.append(sto_domingo_tv)
czo4_tv = pd.concat(list_czo4)
czo4_tv.reset_index(drop=True, inplace=True)

In [336]:
czo4_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Manta,55.25,0,1
1,Manta,67.25,10,2
2,Manta,77.25,1,11
3,Manta,175.25,0,12
4,Manta,181.25,3,9


In [338]:
name_file = '06_Archivos_intermedios/czo4_tv.xlsx'
sheet_name = 'CZO4'

In [339]:
czo4_tv.to_excel(name_file, sheet_name=sheet_name)

# CZO5

In [160]:
main_path='05_ReportesPromedio_TV/2022/CZ05/CSV files/'
name_file='Unidad_scc-l05_2021-01-01-2022-01-01.csv'

In [161]:
datos = datosFechas(main_path=main_path, name_file=name_file, name_column_fecha='Frecuencia')

In [162]:
datos.groupby('Ciudad').count()

,Frecuencia,Intensidad avg,Ancho de banda avg,Date,Year,Month,Day
Ciudad,,,,,,,
Guayaquil,1358,1358,1358,1358,1358,1358,1358
Quevedo,1860,1860,1860,1860,1860,1860,1860


In [167]:
data1 = datos.groupby('Ciudad').get_group('Quevedo')
list_frec = []
for frec, group_data1 in data1.groupby('Frecuencia'):
    list_frec.append(frec)
print(list_frec)

[55.25, 57.0, 59.75, 61.25, 63.0, 65.75, 67.25, 69.0, 71.75, 77.25, 79.0, 81.75, 83.25, 85.0, 87.5, 87.7, 87.75, 87.9, 88.0, 174.0, 175.25, 177.0, 181.25, 183.0, 187.25, 189.0, 193.25, 195.0, 199.25, 201.0, 205.25, 207.0, 211.25, 213.0, 216.0, 512.0, 513.25, 515.0, 515.14, 519.25, 521.0, 521.14, 525.25, 527.0, 527.14, 531.25, 533.0, 533.14, 537.25, 539.0, 539.14, 543.25, 545.0, 545.14, 549.25, 551.0, 551.14, 555.25, 557.0, 557.14, 561.25, 563.0, 563.14, 567.25, 569.0, 569.14, 573.25, 575.0, 575.14, 579.25, 581.0, 581.14, 585.25, 587.0, 587.14, 591.25, 593.0, 593.14, 597.25, 599.0, 599.14, 603.25, 605.0, 605.14, 609.25, 611.0, 611.14, 615.25, 617.0, 617.14, 621.25, 623.0, 623.14, 627.25, 629.0, 629.14, 633.25, 635.0, 635.14, 639.25, 641.0, 641.14, 645.25, 647.0, 647.14, 651.25, 653.0, 653.14, 657.25, 659.0, 659.14, 663.25, 665.0, 665.14, 669.25, 671.0, 671.14, 675.25, 677.0, 677.14, 681.25, 683.0, 683.14]


De los datos obtenidos se observa lo siguiente:
 * Banda I: portadoras de video, audio y frecuencia central.
     * 55.25, 57.0, 59.75 $\cdots$
     * 83.25, 85.0, 87.5, 87.7, 87.75, 87.9, 88.0 $\cdots$
 * Banda III: portadoras de video y frecuencia central.
     * 174.0, 175.25, 177.0 $\cdots$
     * 181.25, 183.0 $\cdots$
     * 216.0 (se encuentra esta frecuencia que pertenece al extremo derecho del rango de frecuencias para el canal 13)
 * Banda IV: portadora de video y frecuencia central.
     * 513.25, 515.0, 515.14 $\cdots$
 * Banda V: portadora de video y frecuencia central.
     * 615.25, 617.0, 617.14 $\cdots$

## Número de meses de operación TV Guayaquil

In [166]:
guayaquil_tv = estadoOperacionTV(data=data1, ciudad='Guayaquil')
guayaquil_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Guayaquil,55.25,11,0
1,Guayaquil,59.75,10,1
2,Guayaquil,61.25,0,11
3,Guayaquil,65.75,0,11
4,Guayaquil,67.25,5,6


## Número de meses de operación TV Quevedo

In [168]:
quevedo_tv = estadoOperacionTV(data=data1, ciudad='Quevedo')
quevedo_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Quevedo,55.25,0,15
1,Quevedo,59.75,0,15
2,Quevedo,61.25,0,15
3,Quevedo,65.75,0,15
4,Quevedo,67.25,0,15


Para Quevedo se observan posibles datos duplicados en los valores de los promedios mensuales.
* Febrero, Marzo, Abril, Mayo, Diciembre

In [301]:
quevedo_tv.to_excel('06_Archivos_intermedios/Duplicados/quevedo_tv_v1.xlsx')

### Datos Quevedo - wrangling

In [302]:
name_file = '06_Archivos_intermedios/Duplicados/Unidad_scc-l03_2021-01-01-2022-01-01_v1.xlsx'

In [303]:
data = pd.read_excel(name_file, index_col=False)
data.head()

,Fecha,Frecuencia,Intensidad avg,Ancho de banda avg,Unidad
0,2021-01-01 00:00:00-05:00,55.25,80.31,4458.19,scc-l02
1,2021-01-01 00:00:00-05:00,57.00,40.98,4463.64,scc-l02
2,2021-01-01 00:00:00-05:00,59.75,68.94,4473.88,scc-l02
3,2021-01-01 00:00:00-05:00,61.25,30.20,878.00,scc-l02
4,2021-01-01 00:00:00-05:00,63.00,34.22,7634.49,scc-l02


In [304]:
data.groupby('Unidad').count()

,Fecha,Frecuencia,Intensidad avg,Ancho de banda avg
Unidad,,,,
scc-l02,1354,1354,1354,1354
scc-l03,1677,1677,1677,1677


In [305]:
data = data.groupby('Unidad').get_group('scc-l03')
data.head()

,Fecha,Frecuencia,Intensidad avg,Ancho de banda avg,Unidad
1354,2021-01-01 00:00:00-05:00,55.25,45.88,9993.74,scc-l03
1355,2021-01-01 00:00:00-05:00,57.00,43.80,9952.61,scc-l03
1356,2021-01-01 00:00:00-05:00,59.75,43.85,9824.21,scc-l03
1357,2021-01-01 00:00:00-05:00,61.25,56.14,9875.74,scc-l03
1358,2021-01-01 00:00:00-05:00,63.00,44.29,9707.74,scc-l03


In [306]:
data.loc[data['Unidad'] == 'scc-l03', 'Ciudad'] = 'Quevedo'

In [307]:
data.head(2)

,Fecha,Frecuencia,Intensidad avg,Ancho de banda avg,Unidad,Ciudad
1354,2021-01-01 00:00:00-05:00,55.25,45.88,9993.74,scc-l03,Quevedo
1355,2021-01-01 00:00:00-05:00,57.00,43.80,9952.61,scc-l03,Quevedo


In [309]:
data['Date'] = pd.to_datetime(data['Fecha'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day
data.drop(columns=['Fecha', 'Unidad'], inplace=True)

In [326]:
quevedo_tv = estadoOperacionTV(data=data, ciudad='Quevedo')
quevedo_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Quevedo,55.25,0,11
1,Quevedo,59.75,0,11
2,Quevedo,61.25,0,11
3,Quevedo,65.75,0,11
4,Quevedo,67.25,0,11


In [327]:
list_quevedo_tv=[]
list_quevedo_tv.append(guayaquil_tv)
list_quevedo_tv.append(quevedo_tv)
czo5_tv = pd.concat(list_quevedo_tv)
czo5_tv.reset_index(drop=True, inplace=True)

In [328]:
czo5_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Guayaquil,55.25,11,0
1,Guayaquil,59.75,10,1
2,Guayaquil,61.25,0,11
3,Guayaquil,65.75,0,11
4,Guayaquil,67.25,5,6


In [329]:
name_file = '06_Archivos_intermedios/czo5_tv.xlsx'
sheet_name = 'CZO5'

In [330]:
czo5_tv.to_excel(name_file, sheet_name=sheet_name)

# CZO6

In [186]:
main_path = '05_ReportesPromedio_TV/2022/CZ06/CSV files/'
name_file = 'Unidad_scs-l05_2021-01-01-2022-01-01.csv'
name_file1 = 'Unidad_scc-l04_2021-01-01-2022-01-01.csv'

In [187]:
datos = datosFechas(main_path=main_path, name_file=name_file1, name_column_fecha='Frecuencia')

In [188]:
datos.groupby('Ciudad').count()

,Frecuencia,Intensidad avg,Ancho de banda avg,Date,Year,Month,Day
Ciudad,,,,,,,
Canar,492,492,492,492,492,492,492
Cuenca,387,387,387,387,387,387,387
Loja,492,492,492,492,492,492,492
Macas,492,492,492,492,492,492,492
Machala,492,492,492,492,492,492,492
Zamora,492,492,492,492,492,492,492


In [189]:
datos.groupby('Year')['Month'].count()

Year
2021    2847
Name: Month, dtype: int64

In [190]:
data1 = datos.groupby('Ciudad').get_group('Machala')
list_data1 = []
for frec, group_data1 in data1.groupby('Frecuencia'):
    list_data1.append(frec)
print(list_data1)

[55.25, 61.25, 67.25, 77.25, 83.25, 175.25, 181.25, 187.25, 193.25, 199.25, 205.25, 211.25, 513.25, 519.25, 525.25, 531.25, 537.25, 543.25, 549.25, 555.25, 561.25, 567.25, 573.25, 579.25, 585.25, 591.25, 597.25, 603.25, 609.25, 615.25, 621.25, 627.25, 633.25, 639.25, 645.25, 651.25, 657.25, 663.25, 669.25, 675.25, 681.25]


De los datos obtenidos se observa los siguiente:
* Banda I: Existencia solo de portadoras de video.
    * 55.25, 61.25, 67.25, 77.25, 83.25
* Banda III: Existencia solo de portadoras de video.
    * 175.25, 181.25, 187.25, 193.25, 199.25, 205.25, 211.25
* Banda IV: Existencia solo de portadoras de video.
    * 513.25, 519.25, 525.25, 531.25, 537.25, 543.25 $\cdots$
* Banda V: Existencia solo de portadoras de video.
    * 645.25, 651.25, 657.25, 663.25, 669.25, 675.25, 681.25

## Número de meses de operación TV Cañar (Tambo)

In [177]:
canar_tv = estadoOperacionTV(data=data1, ciudad='Canar')
canar_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Canar,55.25,0,12
1,Canar,61.25,1,11
2,Canar,67.25,0,12
3,Canar,77.25,0,12
4,Canar,83.25,2,10


## Número de meses de operación TV Cuenca (Turi)

In [179]:
cuenca_tv = estadoOperacionTV(data=data1, ciudad='Cuenca')
cuenca_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Cuenca,55.25,10,1
1,Cuenca,61.25,3,6
2,Cuenca,67.25,10,1
3,Cuenca,77.25,9,2
4,Cuenca,83.25,5,4


## Número de meses de operación TV Loja

In [181]:
loja_tv = estadoOperacionTV(data=data1, ciudad='Loja')
loja_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Loja,55.25,12,0
1,Loja,61.25,0,12
2,Loja,67.25,12,0
3,Loja,77.25,11,1
4,Loja,83.25,0,12


## Número de meses de operación TV Macas

In [183]:
macas_tv = estadoOperacionTV(data=data1, ciudad='Macas')
macas_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Macas,55.25,0,12
1,Macas,61.25,3,9
2,Macas,67.25,0,12
3,Macas,77.25,0,12
4,Macas,83.25,0,12


## Número de meses de operación TV Zamora

In [185]:
zamora_tv = estadoOperacionTV(data=data1, ciudad='Zamora')
zamora_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Zamora,55.25,0,12
1,Zamora,61.25,0,12
2,Zamora,67.25,0,12
3,Zamora,77.25,0,12
4,Zamora,83.25,0,12


## Número de meses de operación TV Machala

In [191]:
machala_tv = estadoOperacionTV(data=data1, ciudad='Machala')
machala_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Machala,55.25,0,12
1,Machala,61.25,11,1
2,Machala,67.25,0,12
3,Machala,77.25,0,12
4,Machala,83.25,0,12


In [193]:
list_czo6 = []
list_czo6.append(canar_tv)
list_czo6.append(cuenca_tv)
list_czo6.append(loja_tv)
list_czo6.append(macas_tv)
list_czo6.append(zamora_tv)
list_czo6.append(machala_tv)
czo6_tv = pd.concat(list_czo6)
czo6_tv.reset_index(drop=True, inplace=True)

In [194]:
czo6_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Canar,55.25,0,12
1,Canar,61.25,1,11
2,Canar,67.25,0,12
3,Canar,77.25,0,12
4,Canar,83.25,2,10


In [201]:
name_path_file = '06_Archivos_intermedios/czo6_tv.xlsx'
sheet_name = 'CZO6'
#header=0

In [202]:
czo6_tv.to_excel(name_path_file, sheet_name='CZO6')

# OTG (CZ5G)

In [217]:
main_path = '05_ReportesPromedio_TV/2022/OTG/CSV files/'
name_file = 'Unidad_erm-l02_2021-01-01-2022-01-01.csv'

In [218]:
datos = datosFechas(main_path=main_path, name_file=name_file, name_column_fecha='Fecha')

In [219]:
datos.groupby('Ciudad').count()

,Frecuencia,Intensidad avg,Ancho de banda avg,Date,Year,Month,Day
Ciudad,,,,,,,
San_Cristobal,130,130,130,130,130,130,130
Santa_Cruz,310,310,310,310,310,310,310


In [206]:
data1 = datos.copy()

In [207]:
list_data1 = []
for frec, group_data1 in data1.groupby('Frecuencia'):
    list_data1.append(frec)
print(list_data1)

[55.25, 61.25, 67.25, 77.25, 83.25, 175.25, 181.25, 187.25, 193.25, 199.25, 205.25, 211.25, 513.25, 519.25, 525.25, 531.25, 537.25, 543.25, 549.25, 555.25, 561.25, 567.25, 573.25, 579.25, 585.25, 591.25, 597.25, 603.25]


De los datos obtenidos se observa lo siguiente:
* Banda I: Existencia de solo portadoras de video.
    * 55.25, 61.25, 67.25, 77.25, 83.25
* Banda III: Existencia de solo portadoras de video.
    * 175.25, 181.25, 187.25, 193.25, 199.25, 205.25, 211.25
* Banda IV: Existencia de solo portadoras de video.
    * 513.25, 519.25, 525.25, 531.25 $\cdots$
* Banda V: No hay existencia de portadoras

In [210]:
sta_cruz_tv_banda_I = mesesOperaFrecuenciaTV_banda_I(data=data1, ciudad='Sta_Cruz')
sta_cruz_tv_banda_I.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Sta_Cruz,55.25,0,1
1,Sta_Cruz,61.25,0,1
2,Sta_Cruz,67.25,0,1
3,Sta_Cruz,77.25,0,1
4,Sta_Cruz,83.25,0,1


In [211]:
sta_cruz_tv_banda_III = mesesOperaFrecuenciaTV_banda_III(data=data1, ciudad='Sta_Cruz')
sta_cruz_tv_banda_III.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Sta_Cruz,175.25,0,1
1,Sta_Cruz,181.25,0,1
2,Sta_Cruz,187.25,0,1
3,Sta_Cruz,193.25,0,1
4,Sta_Cruz,199.25,0,1


In [212]:
sta_cruz_tv_banda_IV = mesesOperaFrecuenciaTV_banda_IV(data=data1, ciudad='Sta_Cruz')
sta_cruz_tv_banda_IV.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Sta_Cruz,513.25,0,1
1,Sta_Cruz,519.25,0,1
2,Sta_Cruz,525.25,0,1
3,Sta_Cruz,531.25,0,1
4,Sta_Cruz,537.25,0,1


In [213]:
list_sta_cruz_tv = []
list_sta_cruz_tv.append(sta_cruz_tv_banda_I)
list_sta_cruz_tv.append(sta_cruz_tv_banda_III)
list_sta_cruz_tv.append(sta_cruz_tv_banda_IV)
otg_tv = pd.concat(list_sta_cruz_tv)
otg_tv.reset_index(drop=True, inplace=True)

In [214]:
otg_tv.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Sta_Cruz,55.25,0,1
1,Sta_Cruz,61.25,0,1
2,Sta_Cruz,67.25,0,1
3,Sta_Cruz,77.25,0,1
4,Sta_Cruz,83.25,0,1


In [215]:
name_path = '06_Archivos_intermedios/otg_tv.xlsx'
sheet_name = 'OTG(CZ5G)'

In [216]:
otg_tv.to_excel(name_path, sheet_name=sheet_name)

## Concatenando todos los df con número de meses de operación

In [340]:
czo2_tv.head(2)

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Tulcan,55.25,8,4
1,Tulcan,59.75,0,12


In [341]:
czo3_tv.head(2)

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Ambato,55.25,0,2
1,Ambato,67.25,12,0


In [342]:
czo4_tv.head(2)

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Manta,55.25,0,1
1,Manta,67.25,10,2


In [343]:
czo5_tv.head(2)

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Guayaquil,55.25,11,0
1,Guayaquil,59.75,10,1


In [344]:
czo6_tv.head(2)

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Canar,55.25,0,12
1,Canar,61.25,1,11


In [345]:
otg_tv.head(2)

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Sta_Cruz,55.25,0,1
1,Sta_Cruz,61.25,0,1


In [349]:
appended_data = []
appended_data.append(czo2_tv)
appended_data.append(czo3_tv)
appended_data.append(czo4_tv)
appended_data.append(czo5_tv)
appended_data.append(czo6_tv)
appended_data.append(otg_tv)
appended_data = pd.concat(appended_data)
appended_data.reset_index(drop=True, inplace=True)

In [350]:
appended_data.head()

,Ciudad,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
0,Tulcan,55.25,8,4
1,Tulcan,59.75,0,12
2,Tulcan,61.25,12,0
3,Tulcan,65.75,0,12
4,Tulcan,67.25,12,0


In [351]:
appended_data.index

RangeIndex(start=0, stop=898, step=1)

In [352]:
appended_data.groupby('Ciudad').count()

,Frecuencia,Nro. Meses Opera,Nro. Meses No Opera
Ciudad,,,
Ambato,20,20,20
Canar,41,41,41
Cuenca,41,41,41
Esmeraldas,82,82,82
Guayaquil,46,46,46
Ibarra,82,82,82
Loja,41,41,41
Macas,41,41,41
Machala,41,41,41


In [355]:
name_path = '01_EstacionesEspectra/04_EstadoOperacionControlada_TV/'
name_file = 'EstadoOperacionControladas_TV.xlsx'
sheet_name = 'Nro Meses Operacion'

In [356]:
appended_data.to_excel(os.path.join(name_path, name_file), sheet_name=sheet_name)